In [1]:
import datetime as dt
import sys

from dotenv import load_dotenv, find_dotenv

sys.path.append("../")
import countdowner as cd

load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2


In [2]:
# Read a watchlist

stock_codes = cd.read_watchlist(cd.DATA_DIR / "watchlist.csv")
display(stock_codes)

url = "https://docs.google.com/spreadsheets/d/15bS8R4CGQhOO0izbCyquNf8L-4IeG1xXUfZ8aLEXA0I/edit?usp=sharing"
stock_codes = cd.read_watchlist(url)
display(stock_codes)


['281739', '32467', '297000']

['291156', '32467', '281739']

In [3]:
# Get and parse a product

r = cd.get_product(stock_codes[0])
cd.parse_product(r)

{'stock_code': '291156',
 'name': 'Macro Brazil Nuts ',
 'description': None,
 'size': '250g',
 'unit_price_($)': 3.0,
 'unit_size': '100G',
 'sale_price_($)': 7.5,
 'normal_price_($)': 7.5,
 'discount_(%)': 0.0,
 'datetime': '2020-12-10T10:43:46'}

In [4]:
# Get and parse many products

%time f = cd.collect_products(stock_codes)
f

CPU times: user 74.7 ms, sys: 3.48 ms, total: 78.2 ms
Wall time: 612 ms


,stock_code,name,description,size,unit_price_($),unit_size,sale_price_($),normal_price_($),discount_(%),datetime
0,291156,Macro Brazil Nuts,None,250g,3.00,100G,7.5,7.5,0.0,2020-12-10 10:43:46
1,32467,Green & Blacks Chocolate Block Dark 85% Cocoa,Passionately crafted by bringing together the ...,90g,4.22,100G,3.8,3.8,0.0,2020-12-10 10:43:47
2,281739,Mainland Cheese Block Organic Cheddar,Mainland organic cheddar is a mild cheddar che...,500g,32.00,1KG,16.0,16.0,0.0,2020-12-10 10:43:47


In [5]:
# Filter sales

cd.filter_sales(f)

,name,sale_price_($),normal_price_($),discount_(%)


In [6]:
# Get products in bulk

cd.run_pipeline(cd.DATA_DIR/'watchlist.csv', recipients)

NameError: name 'recipients' is not defined

In [ ]:
# Run pipeline again: read watchlist, collect products, write products to file, 
# filter sales, and email results

cd.run_pipeline(cd.DATA_DIR/'watchlist.csv', out_path=OUT_DIR/'products.csv')
%less {OUT_DIR/'products.csv'}

In [ ]:
# Email sales
recipients = ["brainbummer@mailinator.com"]
print(f"Emailing {recipients}")

username = os.getenv('GMAIL_USERNAME')  # Replace with your GMail username
password = os.getenv('GMAIL_PASSWORD')  # Replace with your GMail password
cd.run_pipeline(cd.DATA_DIR/'watchlist.csv', recipients, gmail_username=username, gmail_password=password)